In [1]:
import os
city = os.getenv("CITY", "moirana")  # default
print("Running pipeline for:", city)

Running pipeline for: moirana


In [2]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
#settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: c:\Users\lulev\Desktop\KTH\mlfs-book


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [3]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [4]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret(f"SENSOR_LOCATION_JSON_{city}").value
param = json.loads(location_str)

today = datetime.date.today()

param

2025-11-18 16:17:22,884 INFO: Initializing external client
2025-11-18 16:17:22,887 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 16:17:24,980 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279155


{'url': 'https://api.waqi.info/feed/@12698',
 'csv_file': 'c:\\Users\\lulev\\Desktop\\KTH\\mlfs-book/data/moirana.csv',
 'latitude': 66.33,
 'longitude': 14.18}

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [5]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name=f'air_quality_{city}',
    version=1,
)
weather_fg = fs.get_feature_group(
    name=f'weather_{city}',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [6]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(param["url"], city, today, AQICN_API_KEY)
aq_today_df

,pm25,city,date,url
0,28.0,moirana,2025-11-18,https://api.waqi.info/feed/@12698


In [7]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   pm25    1 non-null      float32       
 1   city    1 non-null      object        
 2   date    1 non-null      datetime64[ns]
 3   url     1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(2)
memory usage: 156.0+ bytes


In [8]:
project = hopsworks.login()
fs = project.get_feature_store()

2025-11-18 16:17:29,276 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-18 16:17:29,299 INFO: Initializing external client
2025-11-18 16:17:29,306 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 16:17:30,709 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279155


In [9]:
from datetime import timedelta
import pandas as pd

# Only need to lookback one day but incase the day before prediction is missing
# we can still get some values
three_days_ago = today - timedelta(days=3)
three_days_ago_aware = pd.Timestamp(three_days_ago, tz="UTC")

# Tried directly reading with hopswork filter but it shifted the dates one step too much.
df_recent = air_quality_fg.read()

# Filter on last three days
df_recent = df_recent[(df_recent['date'] >= three_days_ago_aware)].sort_values("date")

df_recent

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.00s) 


,date,pm25,city,url,lag1,lag2,lag3,rolling
1334,2025-11-15 00:00:00+00:00,4.0,moirana,https://api.waqi.info/feed/@12698,12.0,11.0,46.0,23.000000
1660,2025-11-16 00:00:00+00:00,27.0,moirana,https://api.waqi.info/feed/@12698,4.0,12.0,11.0,9.000000
546,2025-11-17 00:00:00+00:00,24.0,moirana,https://api.waqi.info/feed/@12698,27.0,4.0,12.0,14.333333
2092,2025-11-18 00:00:00+00:00,27.0,moirana,https://api.waqi.info/feed/@12698,24.0,27.0,4.0,18.333334


In [10]:
two_days_ago_aware = pd.Timestamp(today - timedelta(days=2), tz="UTC")
one_day_ago_aware = pd.Timestamp(today - timedelta(days=1), tz="UTC")

In [11]:
import numpy as np
# Logic such that we can get some data even if one of the last three days are missing.
# Meaning todays lag and rolling feature is only dependent on one days data.
# If a day is missing the rest of the values will still be filled in.
def get_pm25_for_date(df, ts):
    row = df[df["date"] == ts]
    if row.empty:
        return np.nan
    return row["pm25"].values[0]

aq_today_df["lag1"] = get_pm25_for_date(df_recent, one_day_ago_aware)
aq_today_df["lag2"] = get_pm25_for_date(df_recent, two_days_ago_aware)
aq_today_df["lag3"] = get_pm25_for_date(df_recent, three_days_ago_aware)

aq_today_df["rolling"] = aq_today_df[["lag1", "lag2", "lag3"]].mean(axis=1, skipna=True)
aq_today_df


,pm25,city,date,url,lag1,lag2,lag3,rolling
0,28.0,moirana,2025-11-18,https://api.waqi.info/feed/@12698,24.0,27.0,4.0,18.333334


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [12]:
hourly_df = util.get_hourly_weather_forecast(city, param["latitude"], param["longitude"])
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 66.25°N 14.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,-6.55,0.0,4.582052,44.999897,moirana
1,2025-11-19,-11.30,0.0,1.835647,348.690094,moirana
2,2025-11-20,-12.65,0.0,1.800000,90.000000,moirana
3,2025-11-21,-4.30,0.0,3.396233,57.994659,moirana
4,2025-11-22,-8.10,0.0,8.891344,68.629311,moirana
5,2025-11-23,-5.45,0.0,5.804825,60.255207,moirana
6,2025-11-24,-5.10,0.0,6.193674,35.537766,moirana


In [13]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [14]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-18 16:17:35,695 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279155/fs/1265766/fg/1730924


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_moirana_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279155/jobs/named/air_quality_moirana_1_offline_fg_materialization/executions


(Job('air_quality_moirana_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 767004
         }
       },
       "result": {
         "observed_value": 28.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T03:17:35.000695Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_

In [15]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 16:17:50,412 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279155/fs/1265766/fg/1730925


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_moirana_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279155/jobs/named/weather_moirana_1_offline_fg_materialization/executions
2025-11-18 16:18:08,382 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-18 16:18:11,575 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 16:19:41,557 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 16:19:41,724 INFO: Waiting for log aggregation to finish.
2025-11-18 16:19:50,366 INFO: Execution finished successfully.


(Job('weather_moirana_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 767006
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T03:17:50.000412Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
